In [451]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, InputLayer, GRU, Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.losses import MeanAbsoluteError, MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product

import plotly.graph_objects as go
import matplotlib.pyplot as plt

parameters = {
    "dataset":{
        "path": "../data/Processed_Data/Demand_Dataset.csv",
        "trainingSize": .70,
        "validationSize": .15,
        "testSize": .15
    },
    "backtesting":{
        "steps": 96,
        "fixedTrainSize": False,
        "refit": False,
    },
    "validation": {
        "n_splits" : 10
    },
    "lstm":{
        "window_size": 5,
        "epochs": 10,
        "batch_size": 64,
        "n_blocks": 1
    }
}

In [434]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5592019397343805367
xla_global_id: -1
]


In [435]:
#Read df
df = pd.read_csv(parameters['dataset']['path'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70080 entries, 0 to 70079
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         70080 non-null  object 
 1   Demand                       70080 non-null  int64  
 2   Temperature                  70080 non-null  float64
 3   Relative_Humidity            70080 non-null  float64
 4   Precipitation_Total          70080 non-null  float64
 5   Sunshine_Duration            70080 non-null  float64
 6   Shortwave_Radiation          70080 non-null  float64
 7   Direct_Shortwave_Radiation   70080 non-null  float64
 8   Diffuse_Shortwave_Radiation  70080 non-null  float64
 9   Year                         70080 non-null  int64  
 10  Month                        70080 non-null  int64  
 11  Hour                         70080 non-null  int64  
 12  DayOfWeek                    70080 non-null  int64  
 13  DayOfYear       

In [436]:
#Drop Columns
dates = df["Date"]
df = df.drop("Date", axis = 1)

In [437]:
#Change Data Types
def parseData(df):
    for col in df:
        df[col] = df[col].astype(float)
    return df

#Convert to float
df = parseData(df)

In [438]:
y = df["Demand"]
X = df.drop("Demand", axis = 1)
X, y = X.to_numpy(), y.to_numpy()

#Split Dataset
trainingSize = int(parameters["dataset"]["trainingSize"] * X.shape[0])
validationSize = int(parameters["dataset"]["validationSize"] * X.shape[0]) + trainingSize

X_train, y_train = X[:trainingSize], y[:trainingSize]
X_val, y_val = X[trainingSize : validationSize], y[trainingSize : validationSize]
X_test, y_test = X[validationSize:], y[validationSize:]

X_train_dates = dates.loc[:trainingSize]
X_validation_dates = dates.loc[trainingSize:validationSize]
X_test_dates = dates.loc[validationSize:]    

#Reshape Datasets
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], 1))
y_val = y_val.reshape((y_val.shape[0], 1))
y_test = y_test.reshape((y_test.shape[0], 1))

print(f"Training ==> X Shape: {X_train.shape}, y Shape: {y_train.shape}")
print(f"Validation ==> X Shape: {X_val.shape}, y Shape: {y_val.shape}")
print(f"Test ==> X Shape: {X_test.shape}, y Shape: {y_test.shape}")

Training ==> X Shape: (49056, 20, 1), y Shape: (49056, 1)
Validation ==> X Shape: (10512, 20, 1), y Shape: (10512, 1)
Test ==> X Shape: (10512, 20, 1), y Shape: (10512, 1)


In [439]:
#Model
def get_LSTM_Model(num_features, n_units):
    model = Sequential()
    model.add(InputLayer((num_features, 1)))
    model.add(LSTM(n_units, activation='relu', kernel_initializer='glorot_normal', return_sequences=False))
    #model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='linear'))
    return model

def get_GRU_Model(num_features, n_units):
    model = Sequential()
    model.add(InputLayer((num_features, 1)))
    model.add(GRU(n_units, activation='relu', kernel_initializer='glorot_normal', return_sequences=False))
    #model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='linear'))
    return model

#model = get_LSTM_Model(X_test.shape[1], 64)
model = get_GRU_Model(X_train.shape[1], 64)
model.summary()

Model: "sequential_76"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_17 (GRU)                    │ (None, 64)             │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_136 (Dense)               │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,929 (50.50 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

In [440]:
cp = ModelCheckpoint('model.keras', save_best_only=True, monitor='val_root_mean_squared_error', mode='max', verbose=False)
model.compile(
    loss=MeanSquaredError(), 
    optimizer=Adam(learning_rate=0.0001), 
    metrics=[
        MeanAbsoluteError(),
        RootMeanSquaredError()]
)

In [441]:
train_history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val), 
    epochs=5,
    batch_size = parameters["lstm"]["batch_size"],
    callbacks=[cp]
)

Epoch 1/5
767/767 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 2087.7747 - mean_absolute_error: 33.3151 - root_mean_squared_error: 45.4980 - val_loss: 1624.1218 - val_mean_absolute_error: 27.1863 - val_root_mean_squared_error: 40.3004
Epoch 2/5
767/767 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 1504.0250 - mean_absolute_error: 26.9836 - root_mean_squared_error: 38.7782 - val_loss: 1540.9209 - val_mean_absolute_error: 25.4621 - val_root_mean_squared_error: 39.2546
Epoch 3/5
767/767 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 1424.5859 - mean_absolute_error: 25.8335 - root_mean_squared_error: 37.7434 - val_loss: 1530.8539 - val_mean_absolute_error: 25.7756 - val_root_mean_squared_error: 39.1261
Epoch 4/5
767/767 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 1386.5421 - mean_absolute_error: 25.3447 - root_mean_squared_error: 37.2328 - val_loss: 1503.2749 - val_mean_absolute_error: 26.0599 - val_root_mean_squared_error: 38.7721
Epoch 5/5
767/767 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 1360.5237 - mean_

In [442]:
test_history = model.evaluate(
    x = X_test,
    y = y_test,
    batch_size = parameters["lstm"]["batch_size"],
)
predictions = model.predict(
    x = X_test,
    batch_size = parameters["lstm"]["batch_size"]
)

165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1100.3940 - mean_absolute_error: 22.5370 - root_mean_squared_error: 32.9303
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [443]:
def plotPredictions(dates, y_pred, y_test):
    fig = go.Figure()
    trace1 = go.Scatter(x=dates, y=y_test, name="test", mode="lines")
    trace2 = go.Scatter(x=dates, y=y_pred, name="predictions", mode="lines")
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.update_layout(
        title="Real value vs Predicted in Test Data",
        xaxis_title="Date Time",
        yaxis_title="Demand",
        width=1020,
        height=450,
        margin = dict(l=70, r=20, t=55, b=20),
        legend = dict(
            orientation = "h",
            yanchor="top",
            y=1.1,
            xanchor="left",
            x=0.76
        )
    )
    fig.show()

plotPredictions(X_test_dates, predictions.flatten(), y_test.flatten())

In [444]:
def plotLosses(train_loss, validation_loss):
    epochs_list = np.arange(0, 50)
    fig = go.Figure()
    trace1 = go.Scatter(x=epochs_list, y=train_loss, name="Training Loss", mode="lines")
    trace2 = go.Scatter(x=epochs_list, y=validation_loss, name="Validation Loss", mode="lines")
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.update_layout(
        title = "Training and Validation Losses",
        xaxis_title = "Epoch",
        yaxis_title = "MSE Loss Value",
        width = 800,
        height = 400,
        margin = dict(l=40, r=30, t=50, b=30),
        legend = dict(
            orientation = "h",
            yanchor = "top",
            y = .98,
            xanchor = "right",
            x = .98
        ) 
    )
    fig.show()

plotLosses(train_history.history["loss"], train_history.history["val_loss"])

In [445]:
def plotMetrics(train_rmse, validation_rmse, train_mae, validation_mae):
    epochs_list = np.arange(0, 50)
    fig = go.Figure()
    trace1 = go.Scatter(x=epochs_list, y=train_rmse, name="Training RMSE", mode="lines")
    trace2 = go.Scatter(x=epochs_list, y=validation_rmse, name="Validation RMSE", mode="lines")
    trace3 = go.Scatter(x=epochs_list, y=train_mae, name="Training MAE", mode="lines")
    trace4 = go.Scatter(x=epochs_list, y=validation_mae, name="Validation MAE", mode="lines")

    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.add_trace(trace3)
    fig.add_trace(trace4)
    fig.update_layout(
        title = "Metrics in Training/Validation",
        xaxis_title = "Epoch",
        yaxis_title = "RMSE / MAE",
        width = 800,
        height = 400,
        margin = dict(l=40, r=30, t=50, b=30),
        legend = dict(
            orientation = "h",
            yanchor = "top",
            y = .98,
            xanchor = "right",
            x = .98
        ) 
    )
    fig.show()
plotMetrics(train_history.history["root_mean_squared_error"], train_history.history["val_root_mean_squared_error"], train_history.history["mean_absolute_error"], train_history.history["val_mean_absolute_error"])

In [446]:
from sklearn.model_selection import ParameterGrid

def selectModels(model_params, data, parameters):
    X_train, y_train = data["train"]["X"], data["train"]["y"]
    X_val, y_val = data["validation"]["X"], data["validation"]["y"]
    X_test, y_test = data["test"]["X"], data["test"]["y"]
    results = pd.DataFrame(columns=["Model", "nUnits", "nLayers", "dropout", "Loss_All", "MAE", "RMSE"])
    for param in tqdm(ParameterGrid(model_params), total=len(list(ParameterGrid(model_params)))):
        units = param.get('nUnits')
        layers = param.get("nLayers")
        dropout = param.get("dropout")
        days = param.get("days")
        #Build LSTM Model
        lstm_model = Sequential()
        for _ in range(layers - 1):
            lstm_model.add(LSTM(units=units, return_sequences=True, dropout=dropout))
        lstm_model.add(LSTM(units=units, dropout=dropout))
        lstm_model.add(Dense(1))

        #Compile LSTM Model
        lstm_model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[MeanAbsoluteError(), RootMeanSquaredError()])
        lstm_model.fit(X_train, y_train, epochs = parameters["lstm"]["epochs"], batch_size = parameters["lstm"]["batch_size"], validation_data=(X_val, y_val), verbose=False)

        #Eval LSTM Model
        lstm_loss, lstm_mae, lstm_rmse = lstm_model.evaluate(
            x = X_test,
            y = y_test,
            batch_size = parameters["lstm"]["batch_size"],
            verbose=False
        )
        results.loc[len(results.index)] = ['LSTM', units, layers, dropout, lstm_loss, lstm_mae, lstm_rmse] 

        #Build GRU Model
        gru_model = Sequential()
        for _ in range(layers - 1):
            gru_model.add(GRU(units=units, return_sequences=True, dropout=dropout))
        gru_model.add(GRU(units=units, dropout=dropout))
        gru_model.add(Dense(1))

        #Compile GRU Model
        gru_model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[MeanAbsoluteError(), RootMeanSquaredError()])
        gru_model.fit(X_train, y_train, epochs = parameters["lstm"]["epochs"], batch_size = parameters["lstm"]["batch_size"], validation_data=(X_val, y_val), verbose=False)

        #Eval GRU Model
        gru_loss, gru_mae, gru_rmse = gru_model.evaluate(
            x = X_test,
            y = y_test,
            batch_size = parameters["lstm"]["batch_size"],
            verbose=False
        )
        results.loc[len(results.index)] = ['GRU', units, layers, dropout, gru_loss, gru_mae, gru_rmse] 

    return results.sort_values(by=["RMSE", "MAE"])


data = {
    "train" : { "X" : X_train, "y" : y_train },
    "validation": { "X" : X_val, "y" : y_val },
    "test": { "X" : X_test, "y": y_test}
}

models_params = {
    'nUnits': [128],     
    'nLayers': [1, 2, 3],        
    'dropout': [0.0, 0.1, 0.2]
}

results = selectModels(models_params, data, parameters)
results

100%|██████████| 27/27 [2:40:17<00:00, 356.21s/it]  


,Model,nUnits,nLayers,dropout,Loss,MAE,RMSE
17,GRU,128,3,0.0,1194.991821,23.087776,34.568653
11,GRU,128,2,0.0,1208.354492,23.815466,34.761395
4,LSTM,128,1,0.0,1248.276123,24.841604,35.330952
34,LSTM,128,3,0.1,1256.626343,24.116083,35.448925
29,GRU,128,2,0.1,1269.004150,23.886051,35.623085
10,LSTM,128,2,0.0,1273.986450,23.519262,35.692947
16,LSTM,128,3,0.0,1294.812500,23.678059,35.983501
46,LSTM,128,2,0.2,1300.909424,23.865238,36.068123
41,GRU,128,1,0.2,1321.035156,24.956335,36.346046
52,LSTM,128,3,0.2,1326.192139,24.328402,36.416920


In [452]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * np.exp(-0.1)
scheduler_callback = LearningRateScheduler(scheduler)

model = Sequential()
model.add(InputLayer((X_train.shape[1], 1)))
#model.add(GRU(128, activation='relu', kernel_initializer='glorot_normal', return_sequences=True))
#model.add(GRU(128, activation='relu', kernel_initializer='glorot_normal', return_sequences=True))
model.add(GRU(128, activation='relu', kernel_initializer='glorot_normal'))
model.add(Dense(1, activation='linear'))

model.compile(
    loss=MeanSquaredError(), 
    optimizer=Adam(learning_rate=0.0001), 
    metrics=[
        MeanAbsoluteError(),
        RootMeanSquaredError()
    ]
)
train_history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val), 
    epochs=50,
    batch_size = parameters["lstm"]["batch_size"],
    callbacks=[scheduler_callback]
)
predictions = model.predict(
    x = X_test,
    batch_size = parameters["lstm"]["batch_size"]
)
test_history = model.evaluate(
    x = X_test,
    y = y_test,
    batch_size = parameters["lstm"]["batch_size"],
)
plotPredictions(X_test_dates, predictions.flatten(), y_test.flatten())
print(f"MAE: {test_history.history["mean_absolute_error"]}, RMSE: {test_history.history["root_mean_squared_error"]}")


Epoch 1/50
767/767 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - loss: 2118.4275 - mean_absolute_error: 32.3109 - root_mean_squared_error: 45.7577 - val_loss: 1483.5709 - val_mean_absolute_error: 26.4387 - val_root_mean_squared_error: 38.5172 - learning_rate: 1.0000e-04
Epoch 2/50
767/767 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 1423.1223 - mean_absolute_error: 25.8538 - root_mean_squared_error: 37.7234 - val_loss: 1426.2107 - val_mean_absolute_error: 25.0916 - val_root_mean_squared_error: 37.7652 - learning_rate: 1.0000e-04
Epoch 3/50
767/767 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 1313.1135 - mean_absolute_error: 24.4930 - root_mean_squared_error: 36.2332 - val_loss: 1352.8907 - val_mean_absolute_error: 22.7533 - val_root_mean_squared_error: 36.7817 - learning_rate: 1.0000e-04
Epoch 4/50
767/767 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - loss: 1227.0453 - mean_absolute_error: 23.2120 - root_mean_squared_error: 35.0262 - val_loss: 1263.5437 - val_mean_absolute_error: 23.0199 - val_root_mean_squ

TypeError: type Variable doesn't define __round__ method